In [1]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
from update import update_sql_table
import psycopg2

csv-File importieren.

In [2]:
nrw_2022 = pd.read_csv("data/nrw_2022.csv")
nrw_2022.head(20)
nrw_2022.info()

FileNotFoundError: [Errno 2] No such file or directory: 'data/nrw_2022.csv'

Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [ ]:
nrw_2022.columns = nrw_2022.columns.str.replace(" ", "_")
nrw_2022.columns = nrw_2022.columns.str.lower()
nrw_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
nrw_2022.tail()

,einzelplan,kapitel,gruppe,zählnummer,funktion,ansatz_2022_in_euro,ansatz_2021_in_euro,ist_2020_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
8827,20,900,636,20,18,0,0,"0,00",0,Zusätzliche Alters- und Hinterbliebenenversorg...
8828,20,900,637,0,18,10.000,10.000,"1.040,14",0,Erstattungen von Versorgungsbezügen an Zweckve...
8829,20,900,671,0,18,10.000,10.000,"0,00",0,Sonstige Erstattungen von Versorgungsbezügen
8830,20,900,919,10,851,200.000.000,200.000.000,"273.652.769,81",0,"Zuführung an das Sondervermögen ""Pensionsfonds..."
8831,20,900,919,20,851,5.000.000,5.000.000,"5.008.001,71",0,Zuführung der von Dritten gezahlten Versorgung...


Umbenennung der Namen im Columns

In [ ]:
nrw_2022.rename(columns = {'einzelplan':'ep', 'zählnummer':'counter'}, inplace = True)
nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,ansatz_2021_in_euro,ist_2020_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
0,1,10,119,1,11,20.000,40.000,"16.918,46",0,Vermischte Einnahmen
1,1,10,124,1,11,10.000,40.000,"7.524,00",0,Mieten und Pachten
2,1,10,232,0,11,0,0,"0,00",0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,0,"0,00",0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,2.000,"0,00",0,Schuldendiensthilfen und Erstattungen von Verw...


droppen von columns

In [ ]:
nrw_2022.drop('ist_2020_in_euro', inplace=True, axis=1)

drop von column

In [ ]:
nrw_2022.drop('ansatz_2021_in_euro', inplace=True, axis=1)
nrw_2022.head()


,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,verpflichtungsermächtigung_in_euro,zweckbestimmung
0,1,10,119,1,11,20.000,0,Vermischte Einnahmen
1,1,10,124,1,11,10.000,0,Mieten und Pachten
2,1,10,232,0,11,0,0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,0,Schuldendiensthilfen und Erstattungen von Verw...


drop column

In [ ]:
nrw_2022.drop('verpflichtungsermächtigung_in_euro', inplace=True, axis=1)
nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,zweckbestimmung
0,1,10,119,1,11,20.000,Vermischte Einnahmen
1,1,10,124,1,11,10.000,Mieten und Pachten
2,1,10,232,0,11,0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,Schuldendiensthilfen und Erstattungen von Verw...


check-up

In [ ]:
nrw_2022

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,zweckbestimmung
0,1,10,119,1,11,20.000,Vermischte Einnahmen
1,1,10,124,1,11,10.000,Mieten und Pachten
2,1,10,232,0,11,0,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,Schuldendiensthilfen und Erstattungen von Verw...
...,...,...,...,...,...,...,...
8827,20,900,636,20,18,0,Zusätzliche Alters- und Hinterbliebenenversorg...
8828,20,900,637,0,18,10.000,Erstattungen von Versorgungsbezügen an Zweckve...
8829,20,900,671,0,18,10.000,Sonstige Erstattungen von Versorgungsbezügen
8830,20,900,919,10,851,200.000.000,"Zuführung an das Sondervermögen ""Pensionsfonds..."


column eingefügt mit Jahr/Wert=2022

In [ ]:
nrw_2022.insert(loc=6,
          column='year',
          value=2022)

nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,year,zweckbestimmung
0,1,10,119,1,11,20.000,2022,Vermischte Einnahmen
1,1,10,124,1,11,10.000,2022,Mieten und Pachten
2,1,10,232,0,11,0,2022,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,2022,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,2022,Schuldendiensthilfen und Erstattungen von Verw...


Column hinzugefügt mit state_id/wert= 10 (NRW)

In [ ]:
nrw_2022.insert(loc=7,
          column='state_id',
          value=10)

nrw_2022.head()

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,year,state_id,zweckbestimmung
0,1,10,119,1,11,20.000,2022,10,Vermischte Einnahmen
1,1,10,124,1,11,10.000,2022,10,Mieten und Pachten
2,1,10,232,0,11,0,2022,10,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,2022,10,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,2022,10,Schuldendiensthilfen und Erstattungen von Verw...


check-up

In [ ]:
nrw_2022

,ep,kapitel,gruppe,counter,funktion,ansatz_2022_in_euro,year,state_id,zweckbestimmung
0,1,10,119,1,11,20.000,2022,10,Vermischte Einnahmen
1,1,10,124,1,11,10.000,2022,10,Mieten und Pachten
2,1,10,232,0,11,0,2022,10,Erstattung von Verwaltungsausgaben durch die L...
3,1,10,236,0,11,0,2022,10,Erstattungen von Verwaltungsausgaben von Sozia...
4,1,10,261,0,11,2.000,2022,10,Schuldendiensthilfen und Erstattungen von Verw...
...,...,...,...,...,...,...,...,...,...
8827,20,900,636,20,18,0,2022,10,Zusätzliche Alters- und Hinterbliebenenversorg...
8828,20,900,637,0,18,10.000,2022,10,Erstattungen von Versorgungsbezügen an Zweckve...
8829,20,900,671,0,18,10.000,2022,10,Sonstige Erstattungen von Versorgungsbezügen
8830,20,900,919,10,851,200.000.000,2022,10,"Zuführung an das Sondervermögen ""Pensionsfonds..."


Reihenfolge nach upload_budget angepasst -> wird nocht geändert

In [ ]:
nrw_2022 = nrw_2022[['ep', 'kapitel', 'zweckbestimmung', 'ansatz_2022_in_euro', 'gruppe', 'counter', 'funktion', 'year', 'state_id']]
nrw_2022.head()

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,1,10,Vermischte Einnahmen,20.000,119,1,11,2022,10
1,1,10,Mieten und Pachten,10.000,124,1,11,2022,10
2,1,10,Erstattung von Verwaltungsausgaben durch die L...,0,232,0,11,2022,10
3,1,10,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,0,11,2022,10
4,1,10,Schuldendiensthilfen und Erstattungen von Verw...,2.000,261,0,11,2022,10


check-up

In [ ]:
nrw_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8832 entries, 0 to 8831
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ep                   8832 non-null   int64 
 1   kapitel              8832 non-null   int64 
 2   zweckbestimmung      8832 non-null   object
 3   ansatz_2022_in_euro  8832 non-null   object
 4   gruppe               8832 non-null   int64 
 5   counter              8832 non-null   int64 
 6   funktion             8832 non-null   int64 
 7   year                 8832 non-null   int64 
 8   state_id             8832 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 621.1+ KB


Umwandlung der Werte im Columns um weiteres vorgehen zu ermöglichen 

In [ ]:
nrw_2022['ep'] = nrw_2022['ep'].astype(str)
nrw_2022['ep'] = nrw_2022['ep'].apply(lambda x:x.zfill(2))
nrw_2022['kapitel'] = nrw_2022['kapitel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
nrw_2022['counter'] = nrw_2022['counter'].astype(str)
nrw_2022['counter'] = nrw_2022['counter'].apply(lambda x:x.zfill(2))
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,01,10,Vermischte Einnahmen,20.000,119,01,11,2022,10
1,01,10,Mieten und Pachten,10.000,124,01,11,2022,10
2,01,10,Erstattung von Verwaltungsausgaben durch die L...,0,232,00,11,2022,10
3,01,10,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,00,11,2022,10
4,01,10,Schuldendiensthilfen und Erstattungen von Verw...,2.000,261,00,11,2022,10
...,...,...,...,...,...,...,...,...,...
8827,20,900,Zusätzliche Alters- und Hinterbliebenenversorg...,0,636,20,18,2022,10
8828,20,900,Erstattungen von Versorgungsbezügen an Zweckve...,10.000,637,00,18,2022,10
8829,20,900,Sonstige Erstattungen von Versorgungsbezügen,10.000,671,00,18,2022,10
8830,20,900,"Zuführung an das Sondervermögen ""Pensionsfonds...",200.000.000,919,10,851,2022,10


Kapitel überarbeitet mit nullziffer-> vergelich roh daten 

In [ ]:
nrw_2022['kapitel'] = nrw_2022['kapitel'].apply(lambda x:x.zfill(3))

In [ ]:
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,01,010,Vermischte Einnahmen,20.000,119,01,11,2022,10
1,01,010,Mieten und Pachten,10.000,124,01,11,2022,10
2,01,010,Erstattung von Verwaltungsausgaben durch die L...,0,232,00,11,2022,10
3,01,010,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,00,11,2022,10
4,01,010,Schuldendiensthilfen und Erstattungen von Verw...,2.000,261,00,11,2022,10
...,...,...,...,...,...,...,...,...,...
8827,20,900,Zusätzliche Alters- und Hinterbliebenenversorg...,0,636,20,18,2022,10
8828,20,900,Erstattungen von Versorgungsbezügen an Zweckve...,10.000,637,00,18,2022,10
8829,20,900,Sonstige Erstattungen von Versorgungsbezügen,10.000,671,00,18,2022,10
8830,20,900,"Zuführung an das Sondervermögen ""Pensionsfonds...",200.000.000,919,10,851,2022,10


Summen mit '.' werden ersetzt (tausender punkt)

In [ ]:
nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].str.replace(".", "")

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1603/1298728458.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].str.replace(".", "")


check-up

In [ ]:
nrw_2022

umwandeln in int

In [ ]:
nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].astype(int)

checkp-up

In [ ]:
nrw_2022

alle Werte >=400 werden zu negative Zahlen umgewandelt

In [ ]:
nrw_2022.loc[nrw_2022['gruppe'] >= 400, 'ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro']* -1


letzter check up

In [ ]:
nrw_2022

,ep,kapitel,zweckbestimmung,ansatz_2022_in_euro,gruppe,counter,funktion,year,state_id
0,01,010,Vermischte Einnahmen,20000,119,01,11,2022,10
1,01,010,Mieten und Pachten,10000,124,01,11,2022,10
2,01,010,Erstattung von Verwaltungsausgaben durch die L...,0,232,00,11,2022,10
3,01,010,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,00,11,2022,10
4,01,010,Schuldendiensthilfen und Erstattungen von Verw...,2000,261,00,11,2022,10
...,...,...,...,...,...,...,...,...,...
8827,20,900,Zusätzliche Alters- und Hinterbliebenenversorg...,0,636,20,18,2022,10
8828,20,900,Erstattungen von Versorgungsbezügen an Zweckve...,-10000,637,00,18,2022,10
8829,20,900,Sonstige Erstattungen von Versorgungsbezügen,-10000,671,00,18,2022,10
8830,20,900,"Zuführung an das Sondervermögen ""Pensionsfonds...",-200000000,919,10,851,2022,10


In [ ]:
nrw_2022.rename(columns = {'funktion':'fkz', 'ansatz_2022_in_euro':'amount'}, inplace = True)

In [ ]:
nrw_2022.head()

,ep,kapitel,zweckbestimmung,amount,gruppe,counter,fkz,year,state_id
0,01,010,Vermischte Einnahmen,20000,119,01,11,2022,10
1,01,010,Mieten und Pachten,10000,124,01,11,2022,10
2,01,010,Erstattung von Verwaltungsausgaben durch die L...,0,232,00,11,2022,10
3,01,010,Erstattungen von Verwaltungsausgaben von Sozia...,0,236,00,11,2022,10
4,01,010,Schuldendiensthilfen und Erstattungen von Verw...,2000,261,00,11,2022,10


In [10]:
get_sql_config()
engine = get_engine()

In [ ]:
#push_to_database(nrw_2022, "budget_all", engine, "capstone_public_budgeting")

The budget_all table was imported successfully.


In [10]:
df_ep_kapitel = pd.read_csv('data/ep_kapitel_nrw_22.csv',encoding= 'UTF-8',dtype={'ep':str,'kapitel':str})

In [11]:
df_ep_kapitel['year'] = 2022
df_ep_kapitel['state_id'] = 10
df_ep_kapitel

,ep_txt,ep,kapitel,kapitel_txt,year,state_id
0,Landtag,01,010,Landtag,2022,10
1,Landtag,01,100,Landesbeauftragte für Datenschutz und Informat...,2022,10
2,Landtag,01,900,"Versorgung der Beamtinnen und Beamten, Richte...",2022,10
3,Ministerpräsident,02,010,Ministerpräsident,2022,10
4,Ministerpräsident,02,020,Allgemeine Bewilligungen,2022,10
...,...,...,...,...,...,...
244,Ministerium für Landwirtschaft und Verbraucher...,15,090,Zuschüsse der Europäischen Union (EU),2022,10
245,Ministerium für Landwirtschaft und Verbraucher...,15,100,Landwirtschaftskammer Nordrhein-Westfalen und ...,2022,10
246,Ministerium für Landwirtschaft und Verbraucher...,15,200,Landesforstverwaltung,2022,10
247,Ministerium für Landwirtschaft und Verbraucher...,15,400,Nordrhein-Westfälisches Landgestüt,2022,10


In [9]:
update_sql_table(df_ep_kapitel[['ep','kapitel','kapitel_txt','year','state_id']],'kapitel')
update_sql_table(df_ep_kapitel[['ep','ep_text','year','state_id']],'einzelplaene')

The kapitel table was imported successfully.


Wird eigentlich nicht mehr benötigt

In [32]:
#push_to_database(df_ep_kapitel[['ep','kapitel','kapitel_txt','year','state_id']], "kapitel", engine, "capstone_public_budgeting")

The kapitel table was imported successfully.


In [33]:
#push_to_database(df_ep_kapitel[['ep','ep_txt','year','state_id']].drop_duplicates(),
#                "einzelplaene",
#                engine, 
#                "capstone_public_budgeting"
#                )

The einzelplaene table was imported successfully.
